In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [3]:
ratings_df = pd.read_csv('datasets/goodreads/ratings.csv')
books_df = pd.read_csv('datasets/goodreads/books.csv')

In [6]:
print(ratings_df.shape)
print(ratings_df.user_id.nunique())
print(ratings_df.book_id.nunique())
ratings_df.isna().sum()

(981756, 3)
53424
10000


book_id    0
user_id    0
rating     0
dtype: int64

In [8]:
Xtrain, Xtest = train_test_split(ratings_df, test_size=0.2, random_state=1)
print(f"Shape of train data: {Xtrain.shape}")
print(f"Shape of test data: {Xtest.shape}")

Shape of train data: (785404, 3)
Shape of test data: (196352, 3)


In [9]:
#Get the number of unique entities in books and users columns
nbook_id = ratings_df.book_id.nunique()
nuser_id = ratings_df.user_id.nunique()

In [11]:
#Book input network
input_books = tf.keras.layers.Input(shape=[1])
embed_books = tf.keras.layers.Embedding(nbook_id + 1,15)(input_books)
books_out = tf.keras.layers.Flatten()(embed_books)

#user input network
input_users = tf.keras.layers.Input(shape=[1])
embed_users = tf.keras.layers.Embedding(nuser_id + 1,15)(input_users)
users_out = tf.keras.layers.Flatten()(embed_users)

conc_layer = tf.keras.layers.Concatenate()([books_out, users_out])
x = tf.keras.layers.Dense(128, activation='relu')(conc_layer)
x_out = x = tf.keras.layers.Dense(1, activation='relu')(x)
model = tf.keras.Model([input_books, input_users], x_out)

First 3 lines: create input layer to accept a 1D array of book IDs, create an embedding layer with shape of (num unique books + 1, 15). We add 1 to the number of unique books because the embedding layers need an extra row for books that do not appear in the training dataset, OOV entitites. The second dimension (15) is arbitrary. This can be any number depending on how large we want the embedding layer to be.

Notice that we append the input layer to the end of the book embedding layer. This is the functional API in action. What we are basically saying here is that we want to pass the output of the input layer to the embedding layer.

In the next three lines of code, we do the same thing we did for books, but this time for the users. That is, we create an input that accepts the users as a 1D vector, and then we create the user embeddings, as well.

In the concatenate line, we simply concatenate or join both the books and the user embedding layer together, and then add a single dense layer with 128 nodes on top of it. For the final layer of the network, we use a single node, because we’re predicting the ratings given to each book, and that requires just a single node.

In the last line of code, we use the `tf.keras.Model` class to create a single model from our defined architecture. This model is expecting two input arrays (books and users).

In [12]:
opt = tf.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 15)        150015      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 15)        801375      input_2[0][0]                    
______________________________________________________________________________________________

In [13]:
hist = model.fit([Xtrain.book_id, Xtrain.user_id], 
                 Xtrain.rating, 
                 batch_size=64, 
                 epochs=5, 
                 verbose=1, 
                 validation_data=([Xtest.book_id, Xtest.user_id], Xtest.rating))

Epoch 1/5
12272/12272 [==============================] - 74s 6ms/step - loss: 0.8561 - val_loss: 0.7190
Epoch 2/5
12272/12272 [==============================] - 78s 6ms/step - loss: 0.6849 - val_loss: 0.6976
Epoch 3/5
12272/12272 [==============================] - 84s 7ms/step - loss: 0.6485 - val_loss: 0.6934
Epoch 4/5
12272/12272 [==============================] - 73s 6ms/step - loss: 0.6172 - val_loss: 0.6961
Epoch 5/5
12272/12272 [==============================] - 73s 6ms/step - loss: 0.5860 - val_loss: 0.7156
